In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import string
from nltk.corpus import stopwords
import string
from datetime import timedelta
import numpy as np
import emoji

__Read file with every stock name in NASDAQ and NYSE__

In [2]:
screener=pd.read_csv('nasdaq_screener.csv') 

In [3]:
screener['first_name']=[x.split(" ")[0] for x in screener['Name']]  #take the first word of the name
screener.head(5)

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry;,first_name
0,A,Agilent Technologies Inc. Common Stock,$117.37,0.7300,0.626%,3.522431e+10,United States,1999.0,238458,Capital Goods,Electrical Products;,Agilent
1,AA,Alcoa Corporation Common Stock,$58.8621,3.0221,5.412%,1.085704e+10,NaN,2016.0,1864745,Basic Industries,Metal Fabrications;,Alcoa
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.81,0.0100,0.102%,1.226250e+09,NaN,2021.0,707,Finance,Business Services;,Ares
3,AACG,ATA Creativity Global American Depositary Shares,$1.07,-0.0050,-0.465%,3.381049e+07,China,NaN,943,Miscellaneous,Service to the Health Industry;,ATA
4,AACI,Armada Acquisition Corp. I Common Stock,$9.86,0.0000,0.00%,2.041957e+08,United States,2021.0,3,Finance,Diversified Financial Service;,Armada


__Read WallStreetBets posts (need to insert Fiona part here)__

In [4]:
df=pd.read_csv('reddit_wsb.csv',usecols=['title',"score","comms_num","body","timestamp"])
df.head(5)

,title,score,comms_num,body,timestamp
0,"It's not about the money, it's about sending a...",55,6,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,23,NaN,2021-01-28 21:32:10
2,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,74,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,156,NaN,2021-01-28 21:26:56


__Find stocks talked the most about in WSB__

In [5]:
df2=df.copy()
df2['words']=[x.split(" ") for x in df2['title']]

In [6]:
capital=re.compile(r'\b[A-Z]+\b')

In [7]:
stocks = [capital.findall(df2['title'][i]) for i in range(len(df2))] #find all capital letter blocks in the text
stocks2 = [x for sublist in stocks for x in sublist]

In [8]:
sw_list = stopwords.words('english')+ ['amp',''," "]
vct = CountVectorizer(stop_words=sw_list+
                      ['WE', 'YOU', 'WISH', 'GO', 'IT', 'IS', 'ON', 'ALL', 'ARE', 'CAN', 'FOR', 'UP', 'DO', 'GET', 'ME', 'OUT', 'NOW', 'BE', 'ME', 'UK', 'RE', 'OR'],
                      lowercase=False) 
words = vct.fit_transform(stocks2)
word_counts = pd.DataFrame(words.sum(axis=0))
word_counts.columns = vct.get_feature_names()
word_counts = word_counts.T
word_counts = word_counts[0].sort_values(ascending=False)
word_counts.head(100)

GME        8553
AMC        3381
THE        2628
HOLD       1979
YOLO       1628
           ... 
RETARDS     117
VIAC        115
SQUEEZE     114
BOUGHT      113
MARKET      112
Name: 0, Length: 100, dtype: int64

In [9]:
total_words=list(word_counts.loc[word_counts>100].index)

In [10]:
#mask = (screener['Symbol'].str.len() >= 2)
#symbol_list=list(screener.loc[mask,"Symbol"])
#symbol_list

In [11]:
screener['found']=0
for i in screener.index:
        if screener.Symbol[i] in total_words: #or screener.first_name[i] in total_words:
            screener.loc[i,"found"]=1

__24 companies which have more than 100 mentions in WSB__

In [12]:
screener2=screener.loc[screener.found==1]
screener2

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry;,first_name,found
11,AAL,American Airlines Group Inc. Common Stock,$16.891,0.3210,1.937%,1.097096e+10,United States,NaN,10763741,Transportation,Air Freight/Delivery Services;,American,1
417,AMC,AMC Entertainment Holdings Inc. Class A Common...,$11.4601,-0.3799,-3.209%,5.922816e+09,NaN,2013.0,16441822,Consumer Services,Movies/Entertainment;,AMC,1
423,AMD,Advanced Micro Devices Inc. Common Stock,$92.13,3.4000,3.832%,1.499291e+11,United States,NaN,45183187,Technology,Semiconductors;,Advanced,1
660,ASO,Academy Sports and Outdoors Inc. Common Stock,$36.525,0.7150,1.997%,3.122306e+09,United States,2020.0,264370,Consumer Non-Durables,Recreational Products/Toys;,Academy,1
885,BB,BlackBerry Limited Common Stock,$5.21,0.0200,0.385%,3.003424e+09,Canada,NaN,1623949,NaN,;,BlackBerry,1
1678,CLF,Cleveland-Cliffs Inc. Common Stock,$22.51,0.4900,2.225%,1.181183e+10,United States,NaN,3868203,NaN,;,Cleveland-Cliffs,1
1690,CLNE,Clean Energy Fuels Corp. Common Stock,$4.38,0.1600,3.791%,9.757660e+08,United States,2007.0,1018628,Public Utilities,Natural Gas Distribution;,Clean,1
1694,CLOV,Clover Health Investments Corp. Class A Common...,$2.40,0.0300,1.266%,1.057680e+09,United States,NaN,4335197,Health Care,Medical Specialities;,Clover,1
1937,CRSR,Corsair Gaming Inc. Common Stock,$14.865,-0.1350,-0.90%,1.418120e+09,United States,2020.0,172228,Technology,Computer Manufacturing;,Corsair,1
2155,DD,DuPont de Nemours Inc. Common Stock,$65.12,1.8000,2.843%,3.311525e+10,NaN,2017.0,591415,Capital Goods,Wholesale Distributors;,DuPont,1


__Read the stock price data for all of the 24 companies and clean__

In [13]:
for i in screener2.Symbol:
    locals()[str(i)] = pd.read_csv("HistoricalData"+"_"+str(i)+".csv")

In [14]:
for i in screener2.Symbol:
    locals()[i].Date =pd.to_datetime(locals()[i].Date)

In [15]:
for i in screener2.Symbol:
    locals()[i]=locals()[i].loc[locals()[i].Date>min(df.timestamp)]
    locals()[i]=locals()[i].loc[locals()[i].Date<max(df.timestamp)]

In [16]:
for i in screener2.Symbol:
    locals()[i].iloc[:,1] = locals()[i].iloc[:,1].map(lambda x: x.lstrip('$'))
    locals()[i].iloc[:,3] = locals()[i].iloc[:,3].map(lambda x: x.lstrip('$'))
    locals()[i].iloc[:,4] = locals()[i].iloc[:,4].map(lambda x: x.lstrip('$'))
    locals()[i].iloc[:,5] = locals()[i].iloc[:,5].map(lambda x: x.lstrip('$'))

In [17]:
for i in screener2.Symbol:
    locals()[i].iloc[:,[1,3,4,5]]=locals()[i].iloc[:,[1,3,4,5]].astype(float)
    locals()[i]["Diff"]=locals()[i].iloc[:,1]-locals()[i].iloc[:,3]
    locals()[i]["Diff_Pct"]=locals()[i].Diff/locals()[i].Open*100
    locals()[i]["Var"]=locals()[i].High-locals()[i].Low

__Run sentiment analysis__

In [18]:
sid = SentimentIntensityAnalyzer()

pol_scores_title = [sid.polarity_scores(x)['compound'] for x in df2['title']]
pol_scores_body = [sid.polarity_scores(x)['compound'] for x in df2.loc[df2['body'].notna()]["body"]]

df2['sentiment_title'] = pd.Series(pol_scores_title)
df2['sentiment_body'] = 0

In [19]:
df2['sentiment_body'] = 0
i=0
j=0
while i<len(df2):
    if pd.isna(df2["body"][i]):
        i=i+1
    else:
        df2.loc[i,'sentiment_body']=pol_scores_body[j]
        i=i+1
        j=j+1            
    

In [20]:
df2['timestamp'] =pd.to_datetime(df2['timestamp'])
df2['timestamp']=[df2['timestamp'][i].strftime('%Y-%m-%d') for i in range(len(df2))]
df2

,title,score,comms_num,body,timestamp,words,sentiment_title,sentiment_body
0,"It's not about the money, it's about sending a...",55,6,NaN,2021-01-28,"[It's, not, about, the, money,, it's, about, s...",0.0000,0.0000
1,Math Professor Scott Steiner says the numbers ...,110,23,NaN,2021-01-28,"[Math, Professor, Scott, Steiner, says, the, n...",-0.7034,0.0000
2,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28,"[Exit, the, system]",0.0000,-0.4199
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,74,NaN,2021-01-28,"[NEW, SEC, FILING, FOR, GME!, CAN, SOMEONE, LE...",-0.3397,0.0000
4,"Not to distract from GME, just thought our AMC...",71,156,NaN,2021-01-28,"[Not, to, distract, from, GME,, just, thought,...",0.2235,0.0000
...,...,...,...,...,...,...,...,...
53182,What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...,2021-08-02,"[What, I, Learned, Investigating, SAVA, FUD, S...",-0.4278,-0.9641
53183,"Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...,2021-08-02,"[Daily, Popular, Tickers, Thread, for, August,...",0.4215,0.5040
53184,Hitler reacts to the market being irrational,7398,372,NaN,2021-08-02,"[Hitler, reacts, to, the, market, being, irrat...",-0.3400,0.0000
53185,"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...,2021-08-02,"[Daily, Discussion, Thread, for, August, 02,, ...",0.0000,0.7476


__Calculate sentiment score, which takes into account the times the post has been upvoted__

In [21]:
df2["sentiment_score"]=(df2["sentiment_title"]+df2["sentiment_body"])*(df2["score"])

__Filter reddit posts which talk about these 24 companies and separate in different datasets__

In [22]:
for j in screener2.Symbol:
    locals()["Reddit_"+str(j)] = df2.loc[[str(j) in df2.title[i] for i in range(len(df2))]]

__Group posts by date and calculate aggregate sentiment score. Merge with stock price data__

In [23]:
for j in screener2.Symbol:
    locals()["Group_"+str(j)] = locals()["Reddit_"+str(j)].groupby("timestamp").sum("sentiment_score")
    locals()["Group_"+str(j)]["Date"]=pd.to_datetime(locals()["Group_"+str(j)].index)

In [24]:
for j in screener2.Symbol:
    locals()["Merge_"+str(j)]=locals()[str(j)].merge(locals()["Group_"+str(j)], on="Date", how="outer")
    locals()["Merge_"+str(j)]=locals()["Merge_"+str(j)].loc[locals()["Merge_"+str(j)].sentiment_score.notna()]
    locals()["Merge_"+str(j)]=locals()["Merge_"+str(j)].loc[locals()["Merge_"+str(j)].Diff.notna()]
    locals()["Merge_"+str(j)]=locals()["Merge_"+str(j)].iloc[:,[0,2,6,7,8,13]]

__Calculate the correlation between sentiment score and price difference in one day__

In [25]:
Corr=pd.DataFrame(screener2.Symbol).set_index("Symbol")
Corr["Volume"]=0
Corr["Diff"]=0
Corr["Diff_Pct"]=0
Corr["Var"]=0
Corr["sentiment_score"]=0

In [26]:
for j in screener2.Symbol:
    Corr.loc[str(j)]=locals()["Merge_"+str(j)].corr().iloc[-1,:]
Corr

,Volume,Diff,Diff_Pct,Var,sentiment_score
Symbol,,,,,
AAL,-0.113930,0.557336,0.574784,-0.125105,1
AMC,-0.161098,-0.033902,0.023022,-0.078930,1
AMD,0.313595,0.287271,0.265846,0.210674,1
ASO,0.046519,-0.352224,-0.331938,0.273669,1
BB,-0.060440,0.329213,0.245527,-0.270250,1
CLF,0.637864,-0.217788,-0.210929,0.347098,1
CLNE,0.741127,-0.130686,-0.127498,0.130317,1
CLOV,0.321263,-0.276467,-0.284452,0.319413,1
CRSR,0.551624,0.200493,0.174167,0.083765,1


In [27]:
Corr.mean()

Volume             0.232569
Diff               0.001192
Diff_Pct           0.021191
Var                0.155741
sentiment_score    1.000000
dtype: float64

__Results show the average correlation is about 0: WSB does not affect stock prices?__